#Imports

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Com oficial_feedback 

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset_finalv3.csv').dropna(subset=['feedback_rating','correction_time'])
df = df.query('feedback_rating > 0')
df.reset_index(inplace=True)
df.drop(columns='index',inplace=True)

In [ ]:
df.head()

,tipo_cliente,formato_redacao,start,tamanho_comentario_geral,correction_time,feedback_rating,final_score,proporcao_positiva,qtd_comentarios_especificos,proporcao_negativa,...,media_temporal_aluno,media_temporal_corretor,std_temporal_aluno,std_temporal_corretor,mediana_temporal_aluno,mediana_temporal_corretor,score_medio_temporal_aluno,score_medio_temporal_corretor,score_std_temporal_aluno,diff_score_media
0,Individual,Imagem,2015-04-10 00:51:22,0,2039.0,4.0,680.0,0.0,15,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
1,Individual,Imagem,2015-04-21 12:40:07,287,1522.0,1.0,320.0,0.0,23,1.0,...,NaN,4.00,NaN,0.000000,NaN,4.0,0.0,21.250000,0.0,316.0
2,Individual,Imagem,2015-04-21 14:28:42,232,1207.0,1.0,360.0,0.0,21,1.0,...,NaN,2.50,NaN,1.500000,NaN,2.5,NaN,21.328125,NaN,357.5
3,Individual,Imagem,2015-04-23 18:26:09,0,2322.0,1.0,480.0,0.0,22,1.0,...,NaN,1.75,NaN,1.414214,NaN,1.0,NaN,5.958252,NaN,479.0
4,Individual,Imagem,2015-05-14 14:35:51,259,945.0,4.0,640.0,0.0,26,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.000000,NaN,NaN


In [ ]:
df.isnull().sum()

tipo_cliente                                 0
formato_redacao                              0
start                                        0
tamanho_comentario_geral                     0
correction_time                              0
feedback_rating                              0
final_score                                  0
proporcao_positiva                           0
qtd_comentarios_especificos                  0
proporcao_negativa                           0
qtd_comentarios_positivos                    0
qtd_comentarios_negativos                    0
tamanho_medio_comentarios_especificos        0
tempo_vinculo_corretor                       0
qtd_correcoes_anteriores                     0
qtd_correcoes_dia                            0
tempo_espera                                 0
dia_semana                                   0
mes_ano                                      0
dia_mes                                      0
tempo_vinculo_aluno                          0
theme_id_essa

In [ ]:
df.start = pd.to_datetime(df.start)
df['year'] = [df.loc[i].start.year for i in range(len(df))]
df2020 = df.query('year >= 2020')
df2020.reset_index(inplace=True)
df2020.drop(columns='index',inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df2020.dropna(inplace=True)
df2020.reset_index(inplace=True)
df2020.drop(columns=['index'],inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df2020['feedback_rating'].value_counts()

5.0    16299
1.0     9706
3.0     9389
2.0     6272
4.0     3916
Name: feedback_rating, dtype: int64

In [ ]:
X = df2020.drop(columns = ['feedback_rating','start'])
y = df2020['feedback_rating']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

In [ ]:
!pip install category_encoders

In [ ]:

#aplicando o encoder nas colunas categóricas

import category_encoders as ce 
# parâmetro cols identifica as colunas categóricas
js_treino = ce.JamesSteinEncoder(cols=['tipo_cliente','formato_redacao', 'dia_semana',
                                       'mes_ano','dia_mes','theme_id_essay'], model='beta')



treino = js_treino.fit_transform(X_train, y_train)

teste = js_treino.transform(X_test)


treino = pd.concat([treino,y_train],axis=1)

teste = pd.concat([teste,y_test],axis=1)

In [ ]:
X_train2 = treino.drop(columns='feedback_rating')
y_train2 = treino.feedback_rating

X_test2 = teste.drop(columns='feedback_rating')
y_test2 = teste.feedback_rating


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=123)
X_resampled, y_resampled = rus.fit_resample(X_train2, y_train2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def treinar_modelos(lista_modelos,x_treino,y_treino,x_test,y_test):
  mae = []
  mse = []
  dic_metricas = {'MAE':mae,'MSE':mse}
  for i in range(len(lista_modelos)):
    lista_modelos[i].fit(x_treino,y_treino)
    y_pred = lista_modelos[i].predict(x_test)

    mae.append((str(lista_modelos[i]),mean_absolute_error(y_test,y_pred)))
    mse.append((str(lista_modelos[i]),mean_squared_error(y_test,y_pred)))

  return dic_metricas

In [ ]:
modelos = [RandomForestRegressor(),GradientBoostingRegressor(),AdaBoostRegressor(),LinearRegression()]
resultados = treinar_modelos(modelos,X_resampled,y_resampled,X_test2,y_test2)

In [ ]:
X_resampled.shape

(13220, 32)

In [ ]:
X_test2.shape

(15043, 32)

In [ ]:
resultados

{'MAE': [('RandomForestRegressor()', 0.841848700392209),
  ('GradientBoostingRegressor()', 0.8326094153953528),
  ('AdaBoostRegressor()', 1.0063573020858714),
  ('LinearRegression()', 0.883630382862697)],
 'MSE': [('RandomForestRegressor()', 1.0707546300604933),
  ('GradientBoostingRegressor()', 1.043238007134429),
  ('AdaBoostRegressor()', 1.3363508766652283),
  ('LinearRegression()', 1.1429890489833727)]}

In [ ]:
resultados['MAE']

[('RandomForestRegressor()', 0.841848700392209),
 ('GradientBoostingRegressor()', 0.8326094153953528),
 ('AdaBoostRegressor()', 1.0063573020858714),
 ('LinearRegression()', 0.883630382862697)]

In [ ]:
resultados['MSE']

[('RandomForestRegressor()', 1.0707546300604933),
 ('GradientBoostingRegressor()', 1.043238007134429),
 ('AdaBoostRegressor()', 1.3363508766652283),
 ('LinearRegression()', 1.1429890489833727)]

#Sem oficial_feedback

## Abordagem padrão ( 0 e 1)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset_finalv3.csv').dropna(subset=['feedback_rating','correction_time'])
df = df.query('feedback_rating > 0').drop(columns = ['oficial_feedback_revis'])
df.reset_index(inplace=True)
df.drop(columns='index',inplace=True)

In [ ]:
df.head()

,tipo_cliente,formato_redacao,start,tamanho_comentario_geral,correction_time,feedback_rating,proporcao_positiva,qtd_comentarios_especificos,proporcao_negativa,qtd_comentarios_positivos,...,qtd_feedbacks_aluno,feedbacks_positivos_aluno,feedbacks_negativos_aluno,proporcao_feedback_positivo_aluno,proporcao_feedback_negativo_aluno,qtd_feedbacks_corretor,feedbacks_positivos_corretor,feedbacks_negativos_corretor,proporcao_feedback_positivo_corretor,proporcao_feedback_negativo_corretor
0,Individual,Imagem,2015-04-10 00:51:22,0,2039.0,4.0,0.0,15,1.0,0.0,...,0,0,0,NaN,NaN,0,0,0,0.000000,0.000000
1,Individual,Imagem,2015-04-21 12:40:07,287,1522.0,1.0,0.0,23,1.0,0.0,...,0,0,0,0.0,0.0,1,1,0,1.000000,0.000000
2,Individual,Imagem,2015-04-21 14:28:42,232,1207.0,1.0,0.0,21,1.0,0.0,...,0,0,0,NaN,NaN,2,1,1,0.500000,0.500000
3,Individual,Imagem,2015-04-23 18:26:09,0,2322.0,1.0,0.0,22,1.0,0.0,...,0,0,0,NaN,NaN,3,1,2,0.333333,0.666667
4,Individual,Imagem,2015-05-14 14:35:51,259,945.0,4.0,0.0,26,1.0,0.0,...,0,0,0,NaN,NaN,0,0,0,0.000000,0.000000


In [ ]:
df.isnull().sum()

tipo_cliente                                 0
formato_redacao                              0
start                                        0
tamanho_comentario_geral                     0
correction_time                              0
feedback_rating                              0
proporcao_positiva                           0
qtd_comentarios_especificos                  0
proporcao_negativa                           0
qtd_comentarios_positivos                    0
qtd_comentarios_negativos                    0
tamanho_medio_comentarios_especificos        0
tempo_vinculo_corretor                       0
qtd_correcoes_anteriores                     0
qtd_correcoes_dia                            0
tempo_espera                                 0
dia_semana                                   0
mes_ano                                      0
dia_mes                                      0
tempo_vinculo_aluno                          0
theme_id_essay                               0
competencia1 

In [ ]:
df.start = pd.to_datetime(df.start)
df['year'] = [df.loc[i].start.year for i in range(len(df))]
df2020 = df.query('year >= 2020')
df2020.reset_index(inplace=True)
df2020.drop(columns='index',inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df2020.dropna(inplace=True)
df2020.reset_index(inplace=True)
df2020.drop(columns=['index'],inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df2020['feedback_rating'].value_counts()

5.0    16274
1.0     9587
3.0     9364
2.0     6256
4.0     3907
Name: feedback_rating, dtype: int64

In [ ]:
X = df2020.drop(columns = ['feedback_rating','start'])
y = df2020['feedback_rating']

In [ ]:
import datetime
(df2020.start < datetime.datetime(2021,9,1)).value_counts(normalize=True)*100

True     80.058606
False    19.941394
Name: start, dtype: float64

In [ ]:
train = df2020[(df2020.start < datetime.datetime(2021,9,1))].drop(columns=['start','year'])
test = df2020[~(df2020.start < datetime.datetime(2021,9,1))].drop(columns=['start','year'])

In [ ]:
!pip install category_encoders

     |████████████████████████████████| 80 kB 5.7 MB/s 


In [ ]:

#aplicando o encoder nas colunas categóricas

import category_encoders as ce 
# parâmetro cols identifica as colunas categóricas
js_treino = ce.JamesSteinEncoder(cols=['tipo_cliente','formato_redacao', 'dia_semana',
                                       'mes_ano','dia_mes','theme_id_essay',
                                       ], model='beta')



treino = js_treino.fit_transform(train.drop(columns='feedback_rating'), train.feedback_rating)

teste = js_treino.transform(test.drop(columns='feedback_rating'))


treino = pd.concat([treino,train.feedback_rating],axis=1)

teste = pd.concat([teste,test.feedback_rating],axis=1)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
X_train2 = treino.drop(columns='feedback_rating')
y_train2 = treino.feedback_rating

X_test2 = teste.drop(columns='feedback_rating')
y_test2 = teste.feedback_rating


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=123)
X_resampled, y_resampled = rus.fit_resample(X_train2, y_train2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def treinar_modelos(lista_modelos,x_treino,y_treino,x_test,y_test):
  mae = []
  mse = []
  dic_metricas = {'MAE':mae,'MSE':mse}
  for i in range(len(lista_modelos)):
    lista_modelos[i].fit(x_treino,y_treino)
    y_pred = lista_modelos[i].predict(x_test)

    mae.append((str(lista_modelos[i]),mean_absolute_error(y_test,y_pred)))
    mse.append((str(lista_modelos[i]),mean_squared_error(y_test,y_pred)))

  return dic_metricas

In [ ]:
modelos = [RandomForestRegressor(),GradientBoostingRegressor(),AdaBoostRegressor(),LinearRegression()]
resultados = treinar_modelos(modelos,X_resampled,y_resampled,X_test2,y_test2)

In [ ]:
resultados

{'MAE': [('RandomForestRegressor()', 0.9944591757816816),
  ('GradientBoostingRegressor()', 0.962904011866874),
  ('AdaBoostRegressor()', 1.1622419392775651),
  ('LinearRegression()', 0.9938319719154681)],
 'MSE': [('RandomForestRegressor()', 1.425400784443708),
  ('GradientBoostingRegressor()', 1.3551597011204866),
  ('AdaBoostRegressor()', 1.7504744359846036),
  ('LinearRegression()', 1.4196100070694706)]}

In [ ]:
resultados['MAE']

[('RandomForestRegressor()', 0.9944591757816816),
 ('GradientBoostingRegressor()', 0.962904011866874),
 ('AdaBoostRegressor()', 1.1622419392775651),
 ('LinearRegression()', 0.9938319719154681)]

In [ ]:
resultados['MSE']

[('RandomForestRegressor()', 1.425400784443708),
 ('GradientBoostingRegressor()', 1.3551597011204866),
 ('AdaBoostRegressor()', 1.7504744359846036),
 ('LinearRegression()', 1.4196100070694706)]

##Apenas com o 1

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/Imaginie/BD/dataset_finalv2.csv').dropna(subset=['feedback_rating','correction_time'])
df = df.query('feedback_rating > 0 & oficial_feedback_revis == 1').drop(columns = ['oficial_feedback_revis'])
df.reset_index(inplace=True)
df.drop(columns='index',inplace=True)

In [ ]:
df.head()

,tipo_cliente,formato_redacao,start,tamanho_comentario_geral,correction_time,feedback_rating,final_score,proporcao_positiva,qtd_comentarios_especificos,proporcao_negativa,...,media_temporal_aluno,media_temporal_corretor,std_temporal_aluno,std_temporal_corretor,mediana_temporal_aluno,mediana_temporal_corretor,score_medio_temporal_aluno,score_medio_temporal_corretor,score_std_temporal_aluno,diff_score_media
0,Individual,Imagem,2015-04-10 00:51:22,0,2039.0,4.0,680.0,0.0,15,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
1,Individual,Imagem,2015-04-21 12:40:07,287,1522.0,1.0,320.0,0.0,23,1.0,...,NaN,4.00,NaN,0.000000,NaN,4.0,0.0,21.250000,0.0,316.0
2,Individual,Imagem,2015-04-21 14:28:42,232,1207.0,1.0,360.0,0.0,21,1.0,...,NaN,2.50,NaN,1.500000,NaN,2.5,NaN,21.328125,NaN,357.5
3,Individual,Imagem,2015-04-23 18:26:09,0,2322.0,1.0,480.0,0.0,22,1.0,...,NaN,1.75,NaN,1.414214,NaN,1.0,NaN,5.958252,NaN,479.0
4,Individual,Imagem,2015-05-14 14:35:51,259,945.0,4.0,640.0,0.0,26,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.000000,NaN,NaN


In [ ]:
df.isnull().sum()

tipo_cliente                                 0
formato_redacao                              0
start                                        0
tamanho_comentario_geral                     0
correction_time                              0
feedback_rating                              0
final_score                                  0
proporcao_positiva                           0
qtd_comentarios_especificos                  0
proporcao_negativa                           0
qtd_comentarios_positivos                    0
qtd_comentarios_negativos                    0
tamanho_medio_comentarios_especificos        0
tempo_vinculo_corretor                       0
qtd_correcoes_anteriores                     0
qtd_correcoes_dia                            0
tempo_espera                                 0
dia_semana                                   0
mes_ano                                      0
dia_mes                                      0
tempo_vinculo_aluno                          0
theme_id_essa

In [ ]:
df.start = pd.to_datetime(df.start)
df['year'] = [df.loc[i].start.year for i in range(len(df))]
df2020 = df.query('year >= 2020')
df2020.reset_index(inplace=True)
df2020.drop(columns='index',inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df2020.dropna(inplace=True)
df2020.reset_index(inplace=True)
df2020.drop(columns=['index'],inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
df2020['feedback_rating'].value_counts()

5.0    16287
4.0     3885
1.0     3512
3.0     3145
2.0     2068
Name: feedback_rating, dtype: int64

In [ ]:
X = df2020.drop(columns = ['feedback_rating','start'])
y = df2020['feedback_rating']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

In [ ]:
!pip install category_encoders

In [ ]:

#aplicando o encoder nas colunas categóricas

import category_encoders as ce 
# parâmetro cols identifica as colunas categóricas
js_treino = ce.JamesSteinEncoder(cols=['tipo_cliente','formato_redacao', 'dia_semana',
                                       'mes_ano','dia_mes','theme_id_essay'], model='beta')



treino = js_treino.fit_transform(X_train, y_train)

teste = js_treino.transform(X_test)


treino = pd.concat([treino,y_train],axis=1)

teste = pd.concat([teste,y_test],axis=1)

In [ ]:
X_train2 = treino.drop(columns='feedback_rating')
y_train2 = treino.feedback_rating

X_test2 = teste.drop(columns='feedback_rating')
y_test2 = teste.feedback_rating


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=123)
X_resampled, y_resampled = rus.fit_resample(X_train2, y_train2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def treinar_modelos(lista_modelos,x_treino,y_treino,x_test,y_test):
  mae = []
  mse = []
  dic_metricas = {'MAE':mae,'MSE':mse}
  for i in range(len(lista_modelos)):
    lista_modelos[i].fit(x_treino,y_treino)
    y_pred = lista_modelos[i].predict(x_test)

    mae.append((str(lista_modelos[i]),mean_absolute_error(y_test,y_pred)))
    mse.append((str(lista_modelos[i]),mean_squared_error(y_test,y_pred)))

  return dic_metricas

In [ ]:
modelos = [RandomForestRegressor(),GradientBoostingRegressor(),AdaBoostRegressor(),LinearRegression()]
resultados = treinar_modelos(modelos,X_resampled,y_resampled,X_test2,y_test2)

In [ ]:
X_resampled.shape

(6920, 31)

In [ ]:
X_test2.shape

(9537, 31)

In [ ]:
resultados

{'MAE': [('RandomForestRegressor()', 1.1071542413756945),
  ('GradientBoostingRegressor()', 1.087843460670053),
  ('AdaBoostRegressor()', 1.287340875250823),
  ('LinearRegression()', 1.1235234484122543)],
 'MSE': [('RandomForestRegressor()', 1.6780127398553002),
  ('GradientBoostingRegressor()', 1.6257681984739476),
  ('AdaBoostRegressor()', 2.032770656531925),
  ('LinearRegression()', 1.6796700478758761)]}

In [ ]:
resultados['MAE']

[('RandomForestRegressor()', 1.1071542413756945),
 ('GradientBoostingRegressor()', 1.087843460670053),
 ('AdaBoostRegressor()', 1.287340875250823),
 ('LinearRegression()', 1.1235234484122543)]

In [ ]:
resultados['MSE']

[('RandomForestRegressor()', 1.6780127398553002),
 ('GradientBoostingRegressor()', 1.6257681984739476),
 ('AdaBoostRegressor()', 2.032770656531925),
 ('LinearRegression()', 1.6796700478758761)]